<h2> Segmenting and Clustering Neighborhoods in Toronto <h2>

<h5> Week 3 Part 1 of the Graded Assignment -  </h5>

In [1]:
# Importing the necessary python libraries : 
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Libraries imported.


In [2]:
# Data Scraping - 
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(URL).read().decode('utf-8')
beautiful_soup = BeautifulSoup(page, 'html.parser')

wiki_table = beautiful_soup.body.table.tbody

In [3]:
def GetCell(element):
    cells = element.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

In [4]:
def GetRow():    
    data = []  
    
    for tr in wiki_table.find_all('tr'):
        row = GetCell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [5]:
data = GetRow()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns)
df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df.shape

(180, 3)

In [7]:
df_1 = df[df.Borough != 'Not assigned']
df_1 = df_1.sort_values(by=['Postcode','Borough'])

df_1.reset_index(inplace=True)
df_1.drop('index',axis=1,inplace=True)

df_1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df_postcodes = df_1['Postcode']
df_postcodes.drop_duplicates(inplace=True)
df_2 = pd.DataFrame(df_postcodes)
df_2['Borough'] = '';
df_2['Neighbourhood'] = '';


df_2.reset_index(inplace=True)
df_2.drop('index', axis=1, inplace=True)
df_1.reset_index(inplace=True)
df_1.drop('index', axis=1, inplace=True)

for i in df_2.index:
    for j in df_1.index:
        if df_2.iloc[i, 0] == df_1.iloc[j, 0]:
            df_2.iloc[i, 1] = df_1.iloc[j, 1]
            df_2.iloc[i, 2] = df_2.iloc[i, 2] + ',' + df_1.iloc[j, 2]
            
for i in df_2.index:
    s = df_2.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df_2.iloc[i,2 ] = s

In [9]:
df_2.shape

(103, 3)

<h4> End of Week 3 Part 1 </h4> 